# DATA READING

In [0]:
df = spark.read.format('parquet')\
            .option('inferschema', True)\
            .load('abfss://bronze@cartejdatalake.dfs.core.windows.net/rawdata')

In [0]:
df.display()

# Data Transformation

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df = df.withColumn('model_category',split(col('Model_ID'), '-')[0])
df.display()

In [0]:
df.withColumn('Units_Sold', col('Units_Sold').cast(StringType())).printSchema()

In [0]:
df=df.withColumn('RevPerUnit', col('Revenue')/col('Units_Sold'))
df.display()

# AD-HOC

In [0]:
df.display()

In [0]:
display(df.groupBy('Year','BranchName').agg(sum('Units_Sold').alias('Total_Units')).sort('Year','Total_Units',ascending=[1,0]))

Databricks visualization. Run in Databricks to view.

# DATA WRITING

In [0]:
df.write.format('parquet')\
    .mode('append')\
    .option('path','abfss://silver@cartejdatalake.dfs.core.windows.net/carsales')\
    .save()

# Querying Silver Data

In [0]:
%sql

SELECT * FROM parquet.`abfss://silver@cartejdatalake.dfs.core.windows.net/carsales`